In [9]:
import requests
import os
import torch
import torchvision.models as models
from torchvision import transforms
from diffusers import StableDiffusionPipelineSafe
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
search_url = "https://collectionapi.metmuseum.org/public/collection/v1/search"
params = {
    "q": "Vincent van Gogh",
    "artistOrCulture": True,
}
response = requests.get(search_url, params=params)
data = response.json()

In [3]:
object_ids = data.get("objectIDs", [])
print(f"Found {len(object_ids)} artworks by Van Gogh.")

save_dir = "van_gogh_images"
os.makedirs(save_dir, exist_ok=True)


Found 302 artworks by Van Gogh.


In [4]:
downloaded = 0
for object_id in object_ids:
    if downloaded >= 10:
        break
    
    object_url = f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}"
    object_response = requests.get(object_url)
    object_data = object_response.json()

    image_url = object_data.get("primaryImage")
    title = object_data.get("title", f"van_gogh_{object_id}")

    if image_url:
        print(f"Downloading {title}...")
        img_response = requests.get(image_url)
        img_path = os.path.join(save_dir, f"{title.replace(' ', '_')}.jpg")

        with open(img_path, "wb") as f:
            f.write(img_response.content)

        downloaded += 1

print("Done!")

NameError: name 'object_ids' is not defined

In [12]:
resnet = models.resnet50(pretrained=True)
model = resnet.eval()
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])
transform = transforms.ToTensor()

img = Image.open("van_gogh_images/irises.jpg")
img_tensor = transform(img).unsqueeze(0)


with torch.no_grad():
    features = feature_extractor(img_tensor)

print(features)


tensor([[[[0.1418]],

         [[0.2107]],

         [[0.2022]],

         ...,

         [[0.2474]],

         [[0.4633]],

         [[0.5845]]]])


In [13]:
pipeline = StableDiffusionPipelineSafe.from_pretrained("AIML-TUDA/stable-diffusion-safe")
pipe = pipe.to(device)

feature_extractor = pipe.feature_extractor

features = feature_extractor(image_tensor)
print(features)

Fetching 16 files:  19%|███████████▊                                                   | 3/16 [05:36<24:17, 112.09s/it]


RuntimeError: Data processing error: CAS service error : Error : single flight error: Real call failed: CasObjectError(InternalIOError(Custom { kind: Other, error: reqwest::Error { kind: Decode, source: hyper::Error(Body, Custom { kind: UnexpectedEof, error: IncompleteBody }) } }))